In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


# ขอแก้ 
- `df1 = pd.read_csv(url1,index_col=0,parse_dates=[0])`

เป็น

- `df1 = pd.read_csv(url1,parse_dates=[0])`

In [2]:
url = 'https://raw.githubusercontent.com/Katba-Caroline/Predicting_Indoor_Location_Using_WiFi_Fingerprinting/master/Data/trainingData.csv'
df_temp = pd.read_csv(url,parse_dates=[0])
url1 = 'https://raw.githubusercontent.com/Katba-Caroline/Predicting_Indoor_Location_Using_WiFi_Fingerprinting/master/Data/validationData.csv'
df1_temp = pd.read_csv(url1,parse_dates=[0])

In [3]:
df_temp.head()

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
0,100,100,100,100,100,100,100,100,100,100,...,100,-7541.2643,4.864921e+06,2,1,106,2,2,23,1371713733
1,100,100,100,100,100,100,100,100,100,100,...,100,-7536.6212,4.864934e+06,2,1,106,2,2,23,1371713691
2,100,100,100,100,100,100,100,-97,100,100,...,100,-7519.1524,4.864950e+06,2,1,103,2,2,23,1371714095
3,100,100,100,100,100,100,100,100,100,100,...,100,-7524.5704,4.864934e+06,2,1,102,2,2,23,1371713807
4,100,100,100,100,100,100,100,100,100,100,...,100,-7632.1436,4.864982e+06,0,0,122,2,11,13,1369909710


In [4]:
df_temp.dtypes

WAP001              object
WAP002               int64
WAP003               int64
WAP004               int64
WAP005               int64
                     ...  
SPACEID              int64
RELATIVEPOSITION     int64
USERID               int64
PHONEID              int64
TIMESTAMP            int64
Length: 529, dtype: object

In [5]:
# แก้ไข String เป็น Integer
df_temp['WAP001'] = df_temp['WAP001'].astype(int)

In [6]:
# Check for nan
df_temp.isnull().sum().any()

False

# Choose only 500 rows ขึ้เกียจรอ

In [7]:
df = df_temp.sample(n=500)

In [8]:
FL= np.char.array(df['FLOOR'].values)
BU = np.char.array(df['BUILDINGID'].values)
df['Merge'] = (FL + b'  ' + BU).astype(str)

In [9]:
le = LabelEncoder()

# Fitting
le.fit(df['Merge'])
print(le.classes_)

# Transformation
y_le = le.transform(df['Merge'])
print(y_le)

# Insert columns into origina data
df['Merge_LE'] = y_le

# Display
df[['Merge', 'Merge_LE']]

['0  0' '0  1' '0  2' '1  0' '1  1' '1  2' '2  0' '2  1' '2  2' '3  0'
 '3  1' '3  2' '4  2']
[ 3  6  9  2 12  8  2  2  6  0 11  4  1  8  5  3  1  4  5  8  4  1 12  0
  2  3  9 12  2  6  4  2  0 12  1  5 11 10 11  4  2  8  5  5 11 11  7  6
  4  2  2 11  1  2  7  6  2  7  2  3 11  1  5  2 10 11  6 11 11 11  2  2
  8 12 11  9  7  9  2  2 10  2  9  8  7 12  6  3 12  5 12  8  1  1  2  2
 11 12  6  6  8  6  0 10  1 11  2 10  0  4  9  4  8  4  4  4 11  5  8  8
  2  2 10  5  2  2 12  6  7  5  9  6 11 11  6  2  4  5  6 11  8  2 11  4
 11 11  8 12 12  9  4  9  6  9 12 10  4  9  4  0 11 10  2  4 11  6  6  6
  5  4  2 11 10  8  6  5  2  8  5  5 12  6  1 12 11  9  3 10  5 11  1  5
  2  2  7  5 11  5  4  8  6  4  8 10  5 11 11  8  2  0  8 11  8  8 12  8
  9 10  5  8  5 11  7 11  3  6 10 11  2  1  2  5  1  1 12  9  2  4 12  4
  5 12  4  5  8  6  8 11  4  3  2  5  5  8  7 11  4  2 12  6  6  4  0  7
  1  6  6  7  2  8  2  3  5  4  5 11  6 11  1  0 12  1  1 11  7 10  4  1
  5  7  6  6  1  8  9 12 11  2

,Merge,Merge_LE
17480,1 0,3
10854,2 0,6
10968,3 0,9
18941,0 2,2
1206,4 2,12
...,...,...
1180,4 2,12
2610,3 2,11
11556,1 1,4
147,2 1,7


In [10]:
noin = ['FLOOR','BUILDINGID','Merge']
colsX = [item for item in df.columns if item not in noin] 
X = df[colsX].values
y = df['Merge_LE'].values

In [11]:
X.shape

(500, 528)

In [12]:
y.shape

(500,)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

In [14]:
def calculate_scores(clf, name, data):
        dfa = pd.DataFrame()

        X_train, X_test, y_train, y_test = data
        sc = cross_validate(estimator=clf,
                                X=X_train,
                                y=y_train,
                                cv=5,
                                scoring='accuracy',
                                n_jobs=-1)
         
        ACC = sc['test_score']
        
        data = {'clf': f"{name} (Train)",
                'ACC': f"{ACC.mean():6.3f}",
                'Type': 'Train'}

        dfa = dfa.append(data, ignore_index=True)
        dfa = dfa[['clf', 'ACC', 'Type']]

        # Testing scores
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        ACC = accuracy_score(y_true=y_test, y_pred=y_pred)

        data = {'clf': f"{name} (Test)", 'ACC': ACC,'Type': 'Test'}
        dfa = dfa.append(data, ignore_index=True)
        return dfa

In [15]:
# XGBoost                    
xgb = XGBClassifier(eval_metric="logloss", use_label_encoder=False, n_estimators=100)

# LightGBM
lgb = LGBMClassifier(n_estimators=100)

# CatBoost
cat = CatBoostClassifier(verbose=0, n_estimators=100)

In [16]:
import time

clfs = {'XGBoost': xgb, 'LightGBM': lgb, 'CatBoost': cat }

data =  (X_train, X_test, y_train, y_test)

df_all = []
for name, clf in clfs.items():
    tic = time.perf_counter()
    df_out = calculate_scores(clf, name, data)
    toc = time.perf_counter()
    ep_time = toc - tic
    print(f'Calculating {name}, Elapse Time: {ep_time:0.4f} seconds')
    df_all.append(df_out)

Calculating XGBoost, Elapse Time: 3.4666 seconds
Calculating LightGBM, Elapse Time: 2.4471 seconds
Calculating CatBoost, Elapse Time: 11.0825 seconds


In [17]:
df = pd.concat(df_all, axis=0 )
df = df.set_index('clf')
display(df)

,ACC,Type
clf,,
XGBoost (Train),0.991,Train
XGBoost (Test),1.0,Test
LightGBM (Train),1.000,Train
LightGBM (Test),1.0,Test
CatBoost (Train),1.000,Train
CatBoost (Test),1.0,Test
